In [5]:
# pip install --upgrade torch torchvision torchaudio --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 243.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 234.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 401.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 273.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 429.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 284.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 199.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 271.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 276.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 179.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 142.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from depth_kosmos import requests, DepthKosmosCaptioner
from depth_kosmos import Image as Image_PIL

depth_kosmos_captioner = DepthKosmosCaptioner()


Current directory: /root/Depth-SoM/Depth-Anything-V2



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.p

Using device: cuda


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Using device: cuda


In [1]:
# !pip install numpy --upgrade


In [2]:

url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTp5jMLHnfiO56w8iVWAwI4VvOu4B_5c2C1ww&s"
image = Image_PIL.open(requests.get(url, stream=True).raw)


NameError: name 'Image_PIL' is not defined

In [3]:
# depth_kosmos_captioner.display_depth_images(image)
full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
print(full_caption_string)

NameError: name 'depth_kosmos_captioner' is not defined

In [4]:
import os
import io
import pandas as pd
from PIL import Image as Image_PIL
import shutil
from datasets import load_dataset, Dataset, Image
from tqdm import tqdm
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file


# --- Configuration ---

hf_token = os.getenv("HF_TOKEN")

hf_user = "Rajarshi-Roy-research"
# dataset_name = "Flickr30k_Grounding_Som"
# commit_message = "Updating dataset with image paths"
# img_cols = ["image","wbox_image"]
# image_to_process = "image"

dataset_name = "refcocog-eval-depth"
commit_message = "Updating dataset with images and depth caption"
img_cols = ["image"]
image_to_process = "image"

# --- Helper Functions ---
def save_image_from_bytes(image_data, idx, image_folder):
    """Saves an image from byte data and returns the saved file path."""
    try:
        image_bytes = image_data.get("bytes", None)
        if image_bytes:
            image = Image_PIL.open(io.BytesIO(image_bytes))
            image_path = os.path.join(image_folder, f"image_{idx}.jpg")
            image.save(image_path, format="JPEG")
            print(f"✅ Saved: {image_path}")
            return image_path
        else:
            print(f"⚠️ Skipped row {idx}: No bytes found")
            return None
    except Exception as e:
        print(f"❌ Error saving image at row {idx}: {e}")
        return None

def process_image_from_df(df, depth_kosmos_captioner, image_folder):
    """Processes the DataFrame, adding depth captions and saving progress."""
    if "depth_caption" not in df.columns:
        df["depth_caption"] = None

    for index, row in tqdm(df.iterrows(), total=len(df)):
        if row["depth_caption"] is None:
            try:
                if isinstance(row["image"], dict): #Handle case where image is still a dict
                    image_path = save_image_from_bytes(row["image"], index, image_folder)
                    if image_path:
                        image = Image_PIL.open(image_path)
                        full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
                        df.loc[index, "depth_caption"] = full_caption_string
                elif isinstance(row["image"], str): #Handle case where image is a file path
                    image = Image_PIL.open(row["image"])
                    full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
                    df.loc[index, "depth_caption"] = full_caption_string
                else:
                    print(f"⚠️ Unexpected image type at index {index}: {type(row['image'])}")
                    df.loc[index, "depth_caption"] = "Error: Unexpected image type"

                df.to_parquet("df_with_captions.parquet")
            except Exception as e:
                print(f"Error processing image at index {index}: {e}")
                df.loc[index, "depth_caption"] = "Error"
                df.to_parquet("df_with_captions.parquet")

    return df

def push_df_to_hugging_face(df, img_cols, hf_user, dataset_name, commit_message, hf_token, private=False):
    """Pushes the DataFrame to the Hugging Face Hub."""
    hf_dataset = Dataset.from_pandas(df)
    
    for img_col in img_cols:
        hf_dataset = hf_dataset.cast_column(img_col, Image())

    try:
        hf_dataset.push_to_hub(
            repo_id=f"{hf_user}/{dataset_name}",
            commit_message=commit_message,
            token=hf_token,
            private=private,
        )
        print(f"✅ Dataset successfully uploaded: {hf_user}/{dataset_name}")
    except Exception as e:
        print(f"❌ Error pushing to Hugging Face Hub: {e}")


# --- Main Script ---
# Clean up existing image folder
for img_col in img_cols:
    if os.path.exists(img_col):
        shutil.rmtree(img_col)
    os.makedirs(img_col)

# Load dataset
ds = load_dataset(f"{hf_user}/{dataset_name}")
df = ds["train"].to_pandas()

for img_col in img_cols:
    # Save images and update DataFrame
    for i, row in df.iterrows():
        if isinstance(row[img_col], dict):
            df.at[i, img_col] = save_image_from_bytes(row[img_col], i, img_col)

df.to_csv("updated_dataset.csv", index=False)
print("✅ Updated DataFrame saved as 'updated_dataset.csv'!")

#Process and add captions
try:
    df = pd.read_csv("updated_dataset.csv")
except FileNotFoundError:
    pass

df = process_image_from_df(df, depth_kosmos_captioner, image_to_process)

# Authenticate and push to Hugging Face

login(token=hf_token)
push_df_to_hugging_face(df, img_cols, hf_user, dataset_name, commit_message, hf_token, private=True)

df


README.md:   0%|          | 0.00/715 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/60.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

✅ Saved: image/image_0.jpg
✅ Saved: image/image_1.jpg
✅ Saved: image/image_2.jpg
✅ Saved: image/image_3.jpg
✅ Saved: image/image_4.jpg
✅ Saved: image/image_5.jpg
✅ Saved: image/image_6.jpg
✅ Saved: image/image_7.jpg
✅ Saved: image/image_8.jpg
✅ Saved: image/image_9.jpg
✅ Saved: image/image_10.jpg
✅ Saved: image/image_11.jpg
✅ Saved: image/image_12.jpg
✅ Saved: image/image_13.jpg
✅ Saved: image/image_14.jpg
✅ Saved: image/image_15.jpg
✅ Saved: image/image_16.jpg
✅ Saved: image/image_17.jpg
✅ Saved: image/image_18.jpg
✅ Saved: image/image_19.jpg
✅ Saved: image/image_20.jpg
✅ Saved: image/image_21.jpg
✅ Saved: image/image_22.jpg
✅ Saved: image/image_23.jpg
✅ Saved: image/image_24.jpg
✅ Saved: image/image_25.jpg
✅ Saved: image/image_26.jpg
✅ Saved: image/image_27.jpg
✅ Saved: image/image_28.jpg
✅ Saved: image/image_29.jpg
✅ Saved: image/image_30.jpg
✅ Saved: image/image_31.jpg
✅ Saved: image/image_32.jpg
✅ Saved: image/image_33.jpg
✅ Saved: image/image_34.jpg
✅ Saved: image/image_35.jpg
✅ 

  0%|          | 0/400 [00:00<?, ?it/s]

...

 80%|███████▉  | 319/400 [00:22<00:08,  9.78it/s]

Error processing image at index 317: Numpy is not available
Error processing image at index 318: Numpy is not available
Error processing image at index 319: Numpy is not available
Error processing image at index 320: Numpy is not available


 82%|████████▏ | 328/400 [00:22<00:05, 14.29it/s]

Error processing image at index 321: Numpy is not available
Error processing image at index 322: Numpy is not available
Error processing image at index 323: Numpy is not available
Error processing image at index 324: Numpy is not available
Error processing image at index 325: Numpy is not available
Error processing image at index 326: Numpy is not available
Error processing image at index 327: Numpy is not available
Error processing image at index 328: Numpy is not available
Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/tmp/ipykernel_2581/50784099.py", line 64, in process_image_from_df
    full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
  File "/root/Depth-SoM/depth_kosmos.py", line 141, in get_caption_with_depth
    images = self.depth_context.make_depth_context_img(image)
  File "/root/Depth-SoM/depth_kosmos.py", line 77, in make_depth_context_img
    all_outputs = self.on_submit(image_array)
  File "/root/Depth-SoM/depth_kosmos.py", line 61, in on_submit
    depth = self.predict_depth(image[:, :, ::-1])
  File "/root/Depth-SoM/depth_kosmos.py", line 57, in predict_depth
    return self.model.infer_image(image)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/root/Depth-SoM/Depth-Anything-V2/depth_anything_v2/dpt.py", line 188, in infer_image
    image, (h, w) = self.image2tensor(raw_image, input_size)
  File "/root/Depth-SoM/Dept